In [1]:
import requests
import time
import json
import numpy as np
from tqdm import tqdm
import joblib

In [2]:
BATCH_SIZE = 64

model = "BAAI/bge-m3"  # 查看模型信息 https://cloud.siliconflow.cn/models
api_key = ""  # API密钥

In [3]:
cs_papers = []
with open("arxiv_cs-metadata.json", "r") as fi:
    for line in tqdm(fi):
        data = json.loads(line)
        cs_papers.append(data)


62081it [00:00, 111703.08it/s]


In [4]:
np.random.random((len(cs_papers), 1024)).shape


(62081, 1024)

In [5]:
cs_papers[0]

{'id': '0704.1274',
 'title': 'Parametric Learning and Monte Carlo Optimization',
 'abstract': "  This paper uncovers and explores the close relationship between Monte Carlo\nOptimization of a parametrized integral (MCO), Parametric machine-Learning\n(PL), and `blackbox' or `oracle'-based optimization (BO). We make four\ncontributions. First, we prove that MCO is mathematically identical to a broad\nclass of PL problems. This identity potentially provides a new application\ndomain for all broadly applicable PL techniques: MCO. Second, we introduce\nimmediate sampling, a new version of the Probability Collectives (PC) algorithm\nfor blackbox optimization. Immediate sampling transforms the original BO\nproblem into an MCO problem. Accordingly, by combining these first two\ncontributions, we can apply all PL techniques to BO. In our third contribution\nwe validate this way of improving BO by demonstrating that cross-validation and\nbagging improve immediate sampling. Finally, conventional

In [6]:
def get_embeddings(texts, model, api_key, max_retries=3, sleep_time=20):
    url = "https://api.siliconflow.cn/v1/embeddings"
    payload = {
        "model": model,  # 配置LLM embedding model
        "input": texts,
        "encoding_format": "float"
    }
    headers = {
        "Authorization": api_key,  # 添加siliconflow API 密钥，https://cloud.siliconflow.cn/account/ak
        "Content-Type": "application/json"
    }
    for attempt in range(max_retries):
        response = requests.request("POST", url, json=payload, headers=headers)
        if response.status_code == 200:
            response_embeds = json.loads(response.text)["data"]
            embeddings = [response_embeds[i]["embedding"] for i in range(len(texts))]
            return np.array(embeddings).astype(np.float32)  # (1, 1024)
        elif response.status_code == 429:
            print(f"遇到429错误，正在进行第{attempt+1}次重试...")
            time.sleep(sleep_time)  # 暂停几秒后重试
            continue
        else:
            print(f"Error: {response.status_code}")
            return None
    print(f"达到最大重试次数 {max_retries}，获取embeddings失败")
    return None

In [7]:
# 批量处理数据
embeddings = []  # concat(title, abstract)
titles = []
ids = []
abs = []
for i in tqdm(range(0, len(cs_papers), BATCH_SIZE)):
    batch = cs_papers[i:i + BATCH_SIZE]
    texts = [paper["title"].strip() + " " + paper["abstract"].strip() for paper in batch]

    batch_embeddings = get_embeddings(texts, model, api_key, sleep_time=10)  # ~ 30min
    if batch_embeddings is not None:
        embeddings.extend(batch_embeddings)
        titles.extend([paper["title"] for paper in batch])
        ids.extend([paper["id"] for paper in batch])
        abs.extend([paper["abstract"] for paper in batch])


  0%|          | 4/971 [00:03<12:17,  1.31it/s]

遇到429错误，正在进行第1次重试...


  1%|          | 8/971 [00:16<29:35,  1.84s/it]  

遇到429错误，正在进行第1次重试...


  1%|          | 11/971 [00:29<43:24,  2.71s/it] 

...

 98%|█████████▊| 953/971 [33:00<00:21,  1.22s/it]

遇到429错误，正在进行第1次重试...


 99%|█████████▉| 962/971 [33:18<00:08,  1.04it/s]

遇到429错误，正在进行第1次重试...


100%|██████████| 971/971 [33:34<00:00,  2.08s/it]


In [10]:
len(embeddings), len(titles), len(ids), len(abs)


(970, 62017, 62017, 62017)

In [14]:
embeddings = np.concatenate([item.reshape(-1, 1024) for item in embeddings], 0)


In [ ]:
title + abstract  ==> embedding   # 62000

input: new search query, ==> API ==> embedding

# embedding retrieval
O(N)

# ANN, approximate nearest neighbor  # 向量近似检索


In [15]:
import joblib
joblib.dump((embeddings, titles, ids, abs), "processed_data/embeddings.pkl")


['processed_data/embeddings.pkl']

In [16]:
# import joblib 
embeddings, titles, ids, abs = joblib.load("processed_data/embeddings.pkl")


In [17]:
import faiss  # Facebook AI Search Similarity

nlist = 100  # the number of cells in IndexIVFFlat
k = 20  # the number of cells (out of nlist) that are visited to perform a search
d = 1024  # dimension of embeddings
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist)
# assert not index.is_trained
index.train(embeddings)
# assert index.is_trained

index.add(embeddings)      
index.nprobe = k            # add may be a bit slower as well



In [18]:
# 保存索引
faiss.write_index(index, "processed_data/faiss_index.bin")
